# 0.0 Init

In [1]:
%load_ext autoreload
%autoreload 2

# Importar librerias


In [2]:
import janitor #limpiar datos de manera sencilla
import matplotlib.pyplot as plt # visualizar datos
import missingno # para explorar nuestros valores faltantes
import nhanes.load # cargar datos que vamos a usar
import numpy as np #calculos numericos
import pandas as pd #manejo de dataframes
import scipy.stats #pruebas estadisticas
import seaborn as sns #visualizacion de datos estadisticos
import session_info # para ver que paquetes tenemos 
import sklearn.compose # para realizar imputaciones mas sencillas
import sklearn.impute
import sklearn.preprocessing
import statsmodels.api as sm
import statsmodels.datasets
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.graphics.mosaicplot import mosaic

## Importar funciones personalizadas

In [3]:
%run "../missing_data_imputacion/utils/pandas-missing-extension.ipynb"

## Configurar el aspecto general de las gráficas del proyecto

In [4]:
%matplotlib inline

sns.set(
    rc={
        "figure.figsize": (8, 6)
    }
)

sns.set_style("whitegrid")

## El problema de trabajar con valores faltantes

In [5]:
airquality_df=(
    sm.datasets.get_rdataset("airquality") #cargar el dataset de R 
    .data # accedemos a los datos 
    .clean_names( #limpiamos los nombres de las columnas
         case_type="snake"
     ) 
    .add_column("year",1973) #agregamos una columna con el año
    .assign(
        date=lambda df:pd.to_datetime(df[["year","month","day"]]) # convertimos una variable a una fecha
    )
    .sort_values(by="date") #ordenamos
    .set_index("date") # lo convertimos en un indice
)
airquality_df

/home/sathony/miniforge3/envs/missing_data_imputacion/lib/python3.9/site-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.assign` instead.
  return method(self._obj, *args, **kwargs)


,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


In [6]:
( # ajustar modelos de regresion lineal
    smf.ols( # ajustar un modelo de regresion lineal
    formula="temp ~ ozone", # representada la temperatura en funcion del ozono
    data=airquality_df    # en el dataset airquality_df
    )
    .fit() #ajustar el modelo
    .summary() #resumen del modelo
    .tables[0] #mostrar la tabla 0
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Mon, 11 Mar 2024",Prob (F-statistic):,2.93e-18
Time:,11:48:07,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


In [7]:
(
    smf.ols(
        formula="temp ~ ozone + solar_r", #consideramos variable mas
        data = airquality_df
    )
    .fit()
    .summary()
    .tables[0]
) #comparamos las observaciones, perdemos filas

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Mon, 11 Mar 2024",Prob (F-statistic):,1.47e-16
Time:,11:48:08,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


### Reto: Datos de supervivientes

In [8]:
survival_df = sm.datasets.get_rdataset("flchain", "survival").data

Ajusta algunos modelos utilizando a las variables con valores faltantes como predictoras. ¿Qué sucede?

In [9]:
(
    smf.ols(
        formula="death ~ C(chapter)",
        data = survival_df
    )
    .fit()
    .summary()
    .tables[0]
)

/home/sathony/miniforge3/envs/missing_data_imputacion/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dep. Variable:,death,R-squared:,-inf
Model:,OLS,Adj. R-squared:,-inf
Method:,Least Squares,F-statistic:,-143.5
Date:,"Mon, 11 Mar 2024",Prob (F-statistic):,1.00
Time:,11:48:08,Log-Likelihood:,66256.
No. Observations:,2169,AIC:,-1.325e+05
Df Residuals:,2153,BIC:,-1.324e+05
Df Model:,15,,
Covariance Type:,nonrobust,,
